In [5]:
import sys
from   awq import AutoAWQForCausalLM
import os
import sys
sys.path.append("/home/chenyidong/quant/AutoAWQ")
import awq
os.environ["WORLD_SIZE"] = "1"
import time
import torch
import argparse
import numpy as np
import pandas as pd


if __name__ == "__main__":

    """
    python benchflops.py --model_path /mnt/data/zhongrx/Llama-2-7b-hf --quant_file /mnt/data/zhongrx/Llama-2-7b-hf 

    """
    parser = argparse.ArgumentParser()
    parser.add_argument("--model_path", type=str, default="/mnt/data/zhongrx/Llama-2-7b-hf", help="path to the model")
    parser.add_argument("--quant_file", type=str, default="/mnt/data/chenyd/Llama-2-7b-awq", help="weights filename")
    parser.add_argument("-f", type=str)
    parser.add_argument("--batch_size", type=int, default=1, help="Batch size for cache and generation")
    parser.add_argument("--safetensors", default=False, action="store_true", help="Use for enabling safetensors")
    parser.add_argument("--seq_length", default=128, type=int)

    args = parser.parse_args()

    n_gpus = torch.cuda.device_count()
    max_memory = f'{int(torch.cuda.mem_get_info()[0]/1024**3)-2}GB'
    max_memory = {i: max_memory for i in range(n_gpus)}


    n_generate = args.seq_length
    modelawq = AutoAWQForCausalLM.from_quantized(
        args.model_path, args.quant_file, fuse_layers=True,
        max_new_tokens=n_generate, batch_size=args.batch_size,
        safetensors=args.safetensors
    )

Replacing layers...: 100%|█████████████████████████████████████████████████████████████████████████| 32/32 [00:03<00:00,  8.72it/s]


In [13]:
modelawq.model.model.layers[0]

LlamaDecoderLayer(
  (self_attn): QuantAttentionFused(
    (qkv_proj): WQLinear_GEMM(in_features=4096, out_features=12288, bias=False, w_bit=4, group_size=128)
    (o_proj): WQLinear_GEMM(in_features=4096, out_features=4096, bias=False, w_bit=4, group_size=128)
    (rope): RoPE()
  )
  (mlp): QuantLlamaMLP(
    (down_proj): WQLinear_GEMM(in_features=11008, out_features=4096, bias=False, w_bit=4, group_size=128)
  )
  (input_layernorm): FasterTransformerRMSNorm()
  (post_attention_layernorm): FasterTransformerRMSNorm()
)